---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

def prep():
    
    address = pd.read_csv('addresses.csv', encoding = 'ISO-8859-1' )
    latlons = pd.read_csv('latlons.csv', encoding = 'ISO-8859-1' )
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', low_memory=False )
    test = pd.read_csv('test.csv', encoding = 'ISO-8859-1' , low_memory=False)
    
     #merge all 3 sets
    train = pd.merge(train, address, how='left', left_on='ticket_id', right_on='ticket_id')
    train = pd.merge(train, latlons, how='left', left_on='address', right_on='address')
    
    test = pd.merge(test, address, how='left', left_on='ticket_id', right_on='ticket_id')
    test = pd.merge(test, latlons, how='left', left_on='address', right_on='address')
    
    #set index
    train = train.set_index('ticket_id')
    test = test.set_index('ticket_id')
    
    #dropnull hearing date
    train = train[~train['hearing_date'].isnull()]
    
    #fill 'na' lat and lon
    train.lat.fillna(method='pad', inplace=True)
    train.lon.fillna(method='pad', inplace=True)
    test.lat.fillna(method='pad', inplace=True)
    test.lon.fillna(method='pad', inplace=True)
    
    #remove data that don't exist in test set and all other string (non-valued)
    train = train.drop(['violator_name', 'mailing_address_str_number', 'mailing_address_str_name','non_us_str_code', 
                        'agency_name', 'state', 'disposition',
                        'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code', 
                        'violation_street_name', 'violation_description','country', 'city', 'inspector_name', 
                        'state_fee', 'zip_code', 'violation_street_number','violation_zip_code', 'clean_up_cost', 
                        'payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 
                        'compliance_detail', 'address', 'admin_fee'], axis=1)
    test = test.drop(['violator_name', 'mailing_address_str_number', 'mailing_address_str_name','non_us_str_code', 
                        'agency_name', 'state', 'disposition',
                        'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code' ,
                        'violation_street_name', 'violation_description','country', 'city', 'inspector_name', 
                        'state_fee', 'zip_code', 'violation_street_number','violation_zip_code', 'clean_up_cost', 
                        'address', 'admin_fee'], axis=1)   
    

    #drop null target
    train = train.dropna(subset=['compliance'])
    
    #train = pd.get_dummies(train)
    #divide to features and target
    #X_train
    X = train.drop(['compliance'], axis = 1)
    #y_train
    y = train['compliance']
    

    return X, y, test
    


In [11]:
def blight_model():
    X, y, test = prep()
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    test_scaled = scaler.transform(test)
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 10],
                        alpha=0.001,
                        random_state = 0, 
                        solver='lbfgs', 
                        verbose=0)
    clf.fit(X_scaled, y)

    #clf = KNeighborsClassifier().fit(X_scaled, y)
    predicted = clf.predict_proba(test_scaled)[:,1]
    test['compliance'] = predicted
    
    return test['compliance']

In [12]:
blight_model()

ticket_id
284932    0.039725
285362    0.014407
285361    0.071674
285338    0.044034
285346    0.081133
285345    0.044074
285347    0.108887
285342    0.259190
285530    0.021776
284989    0.039825
285344    0.108342
285343    0.022533
285340    0.021986
285341    0.108902
285349    0.081170
285348    0.044095
284991    0.039823
285532    0.046179
285406    0.035579
285001    0.041057
285006    0.023401
285405    0.014005
285337    0.033428
285496    0.100949
285497    0.040646
285378    0.014726
285589    0.034283
285585    0.039206
285501    0.075314
285581    0.014550
            ...   
376367    0.036145
376366    0.035064
376362    0.033150
376363    0.038620
376365    0.036145
376364    0.035064
376228    0.038774
376265    0.035420
376286    0.270052
376320    0.037666
376314    0.036652
376327    0.290906
376385    0.274770
376435    0.372788
376370    0.291015
376434    0.096320
376459    0.077001
376478    0.004213
376473    0.037030
376484    0.039031
376482    0.033581
37